<a href="https://colab.research.google.com/github/JuyeoungJun/graduation-project/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:

def cleaning(text):
    result = []
    hangle = re.compile('[^ A-Za-zㄱ-ㅣ가-힣]+')
    for word in text.review:
        if type(word) == int:
            continue
        result.append(hangle.sub('', word))
        # result = hangle.findall(word)

    return result


def preprocessing(text):
    # 불용어 지정
    # stop_words = ['저','.','!!'.'ㅎㅎ','ㅋㅋ','~','....','으로','로']

    okt = Okt()
    hannanum = Hannanum()
    kkma = Kkma()

    result = []
    dic = {}

    # log_point = text[3556:]

    log_count = 0

    for word in text.review:
        log_count+=1
        temp = []
        if word == ' ':
          temp.append(0)
          temp.append(0)
          temp.append(0)
          temp.append(0)
          temp.append(0)
          temp.append(0)
          temp.append(0)
          result.append(temp)
          continue
          
        print("log ", log_count, " complete ")
        
        # word = cleaning()
        temp.append(len(word))  # 1.길이
        temp.append(len(kkma.nouns(word)))  # 2.명사 종류의 갯수
        cnt_str = hannanum.pos(word)
        v_count = 0
        n_count = 0
        p_count = 0

        cnt_np = kkma.pos(word)
        np_count = 0
        ec_count = 0
        ignore_count = 0
        # print(word)
        for cnt in cnt_np:
            if (cnt[1] == "NP"):
                np_count += 1  # 5.대명사
            elif (cnt[1] == "EC" or cnt[1] == "ECD" or cnt[1] == "ECE" or cnt[1] == "ECS"):
                ec_count += 1  # 6.접속사
            elif (cnt[1] == "VV" or cnt[1] == "VXV"):
                v_count += 1  # 3.동사
            elif (cnt[1] == "VA" or cnt[1] == "VXA" or cnt[1] == "XR"):
                p_count += 1  # 4.형용사
            elif (cnt[0] == "리뷰" or cnt[0] == "사진"):
                ignore_count += 1  # 7.리뷰관련 단어
        temp.append(v_count)
        temp.append(p_count)
        temp.append(np_count)
        temp.append(ec_count)
        temp.append(ignore_count)
        result.append(temp)
        ## 희소한 단어: DIC 사용
        dic_plus = kkma.pos(word)
        for t in dic_plus:
            if t[0] in dic:
                dic[t[0]] += 1
            else:
                dic[t[0]] = 1

    # 희귀 단어 count
    print(len(text))

    for i in range(len(text)):
        #print(text.review[i])
        if text.review[i] == ' ':
            result[i].append(0)
            continue
        check = kkma.pos(text.review[i])
        unique = 0
        for t in check:
            if t not in dic:
                continue
            if dic[t[0]] <= 3:
                unique += 1
        result[i].append(unique)  # 8.희귀단어

    # 정규화
    trans = np.transpose(result)

    column_max = []
    column_min = []
    column_mean = []
    column_std = []
    for t in trans:
        column_min.append(min(t))
        column_max.append(max(t))
        column_mean.append(np.mean(t))
        column_std.append(np.std(t))
    for i in range(len(result)):
        for j in range(len(result[i])):
            result[i][j] = (result[i][j] - column_min[j]) / (column_max[j] - column_min[j]+0.00001)

    df = pd.DataFrame(result)
    print("store check")
    df.to_csv('/content/drive/My Drive/graduation_project/input/preprocess1.csv', mode='w')
    df.fillna(0)
    return df.fillna(0)

if __name__ == '__main__':
    text = pd.read_csv('/content/drive/My Drive/graduation_project/input/labeling_data.csv', engine='python',
                       encoding='utf-8', names=['review', 'label'])
    #clean = cleaning(text[2500:].fillna(1))
    #clean = pd.DataFrame(clean, columns=["review"])
    #x = preprocessing(clean)
    #learning(x, text.label.fillna(1))

In [ ]:
import numpy as np
import pandas as pd
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Okt
import math
import torch
import torch.nn as nn
import torch.optim as optim
import re
import torch.nn.functional as F
import torch.utils.data as data_utils
from torchvision import transforms
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
class Dataset(data_utils.Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}

    def __len__(self):
        return len(self.X)


class MLPRegressor(nn.Module):

    def __init__(self):
        super(MLPRegressor, self).__init__()
        h1 = nn.Linear(len(X_features), 50)
        h2 = nn.Linear(50, 100)
        h3 = nn.Linear(100, 100)
        h4 = nn.Linear(100, 35)
        h5 = nn.Linear(35, 1)
        h6 = nn.Linear(100,1024)
        h7 = nn.Linear(1024,1024)
        h8 = nn.Linear(1024,100)

        self.hidden = nn.Sequential(
            h1,
            nn.Tanh(),
            h2,
            nn.Tanh(),
            h3,
            nn.Tanh(),
            h6,
            nn.Tanh(),
            h7,
            nn.Tanh(),
            h8,
            nn.Tanh(),
            h4,
            nn.Tanh(),
            h5
            
        )
        if use_cuda:
            self.hidden = self.hidden.cuda()

    def forward(self, x):
        o = self.hidden(x)

        return o

trn = pd.read_csv("/content/drive/My Drive/graduation_project/input/train.csv")
val = pd.read_csv("/content/drive/My Drive/graduation_project/input/vali.csv")
X_features = ["feature_1","feature_2","feature_3","feature_4","feature_5","feature_6","feature_7","feature_8"]
y_feature = ["y"]

trn_X_pd, trn_y_pd = trn[X_features], trn[y_feature]
val_X_pd, val_y_pd = val[X_features], val[y_feature]

trn_X = torch.from_numpy(trn_X_pd.astype(float).values).float()
trn_y = torch.from_numpy(trn_y_pd.astype(float).values).float()

val_X = torch.from_numpy(val_X_pd.astype(float).values).float()
val_y = torch.from_numpy(val_y_pd.astype(float).values).float()


batch_size=128


trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

tmp = next(iter(trn_loader))
num_batches = len(trn_loader)


use_cuda = torch.cuda.is_available()

model = MLPRegressor()
criterion = nn.MSELoss()
learning_rate = 1e-6
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 5000
num_batches = len(trn_loader)

trn_loss_list = []
val_loss_list = []

for epoch in range(num_epochs):
    trn_loss_summary = 0.0
    for i, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        trn_loss = criterion(trn_pred, trn_y)
        trn_loss.backward()
        optimizer.step()

        trn_loss_summary += trn_loss

        if (i + 1) % 15 == 0:
            with torch.no_grad():
                val_loss_summary = 0.0
                for j, val in enumerate(val_loader):
                    val_X, val_y = val['X'], val['y']
                    if use_cuda:
                        val_X, val_y = val_X.cuda(), val_y.cuda()
                    val_pred = model(val_X)
                    val_loss = criterion(val_pred, val_y)
                    val_loss_summary += val_loss

            print("epoch: {}/{} | step: {}/{} | trn_loss: {:.4f} | val_loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, num_batches, (trn_loss_summary / 15) ** (1 / 2),
                (val_loss_summary / len(val_loader)) ** (1 / 2)
            ))

            trn_loss_list.append((trn_loss_summary / 15) ** (1 / 2))
            val_loss_list.append((val_loss_summary / len(val_loader)) ** (1 / 2))
            trn_loss_summval = 0.0

print("finish valing")
plt.figure(figsize=(16,9))
x_range = range(len(trn_loss_list))
plt.plot(x_range, trn_loss_list, label="trn")
plt.plot(x_range, val_loss_list, label="val")
plt.legend()
plt.xlabel("training steps")
plt.ylabel("loss")


##TEST
test = pd.read_csv("/content/drive/My Drive/graduation_project/input/test.csv")

test_X_pd, test_y_pd = test[X_features], test[y_feature]


test_X = torch.from_numpy(test_X_pd.astype(float).values).float()
test_y = torch.from_numpy(test_y_pd.astype(float).values).float()

test = Dataset(test_X, test_y)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

model.eval()
correct = 0
for i, test in enumerate(test_loader):
    test_X, test_y = test['X'], test['y']
    if use_cuda:
        test_X, test_y = test_X.cuda(), test_y.cuda()
    output = model(test_X)
    #print("check",output)
    _, prediction = torch.max(output,1)
    #print(prediction)
    pred = torch.round(output)
    #print(pred)
    correct += pred.eq(test_y).sum()

print('Test set: Accuracy: {:.2f}%'.format(100. * correct / len(test_loader.dataset)))

In [ ]:
!pip install konlpy
